### -- I ) Projet analyse des données immobilière, récupération et traitement des données --

L'objectif du projet est de récupérer des données sur le prix de l'immobilier 
pour afficher l'évolution de son prix par jour et par mois.

Import des librairies

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime
import os

/Users/gj3dgf4hf/Documents/Projet_Python/Figaro-immo/env_Python_3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Vide le dossier appelé "page_html" avant son alimentation.

In [2]:
taile_du_dossier = os.listdir("page_Html")

if(len(taile_du_dossier) > 0):
    for page in range(1,100+1):
        os.remove(f"page_Html/immo{page}.html")
else:
    pass

Requete et récupération des la pages en HTML

In [3]:

#-------------------------------- boucle pour récuperer les pages numérotées.
for page in range(1,100+1):
    # Requete de la page
    url = f'https://immobilier.lefigaro.fr/annonces/immobilier-vente-bien-france.html?page={page}'
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Enregistrement de la page
    file_html = open(f"page_Html/immo{page}.html","w")
    file_html.write(str(soup))

Ouveture du fichier / récupération des données ( prix , type de logement, ville)

In [4]:
l_prix_logement = []
l_ville = []
l_type_de_bien = []

for i in range(1,100+1):
    with open(f"page_Html/immo{i}.html") as file:
        soup2 = BeautifulSoup(file, "html.parser")
    
    print(f"page_Html/immo{i}.html")

    prix_logement = soup2.find_all("span", class_="price")
    ville = soup2.find_all("span", class_="title-location")
    type_de_bien = soup2.find_all("h2", class_="cartouche-liste__title")

    # Add element in list 
    
    for index in range(0, len(prix_logement) ):

        # Nettoyage du prix
        u_prix = prix_logement[index].text
        u_prix = u_prix.replace("€", "")
        u_prix = u_prix.replace(" ", "")
        l_prix_logement.append(u_prix)
       
        # Nettoyage du nom de la ville
        u_ville = ville[index].text
        u_ville = u_ville[:-4]
        l_ville.append(u_ville)

        # Nettoyage du type de bien
        u_type_de_bien = type_de_bien[index].text
        categorie_bien = re.search('[a-zA-Z0-9_]+', u_type_de_bien )
        l_type_de_bien.append(categorie_bien.group())



page_Html/immo1.html
page_Html/immo2.html
page_Html/immo3.html
page_Html/immo4.html
page_Html/immo5.html
page_Html/immo6.html
page_Html/immo7.html
page_Html/immo8.html
page_Html/immo9.html
page_Html/immo10.html
page_Html/immo11.html
page_Html/immo12.html
page_Html/immo13.html
page_Html/immo14.html
page_Html/immo15.html
page_Html/immo16.html
page_Html/immo17.html
page_Html/immo18.html
page_Html/immo19.html
page_Html/immo20.html
page_Html/immo21.html
page_Html/immo22.html
page_Html/immo23.html
page_Html/immo24.html
page_Html/immo25.html
page_Html/immo26.html
page_Html/immo27.html
page_Html/immo28.html
page_Html/immo29.html
page_Html/immo30.html
page_Html/immo31.html
page_Html/immo32.html
page_Html/immo33.html
page_Html/immo34.html
page_Html/immo35.html
page_Html/immo36.html
page_Html/immo37.html
page_Html/immo38.html
page_Html/immo39.html
page_Html/immo40.html
page_Html/immo41.html
page_Html/immo42.html
page_Html/immo43.html
page_Html/immo44.html
page_Html/immo45.html
page_Html/immo46.ht

Création du dataframe contenant les données [ Ville, Type de bien, Prix ].

In [5]:
dataframe = pd.DataFrame({"Ville": l_ville,
                "Type de bien":l_type_de_bien,
                "Prix":l_prix_logement})

Visualisation du dataframe

In [6]:
dataframe.head(5)

,Ville,Type de bien,Prix
0,Nice,appartement,195175
1,Thiais,appartement,288000
2,Couquèques,maison,138000
3,Chalon-sur-saone,appartement,148000
4,La Celle-Saint-Avant,maison,126000


Enregistrement du dataframe au format CSV

In [7]:
dataframe.to_csv("rendu_csv/immo_figaro.csv", index=False)